# E-Commerce Business Analytics Dashboard

## Table of Contents
1. [Introduction & Business Objectives](#introduction)
2. [Data Loading & Configuration](#data-loading)
3. [Data Dictionary](#data-dictionary)
4. [Data Preparation & Quality Assessment](#data-preparation)
5. [Revenue Analysis](#revenue-analysis)
6. [Product Performance Analysis](#product-analysis)
7. [Geographic Performance Analysis](#geographic-analysis)
8. [Customer Experience Analysis](#customer-experience)
9. [Summary of Key Insights](#summary)

---

## 1. Introduction & Business Objectives {#introduction}

This notebook provides a comprehensive analysis of e-commerce business performance focusing on key metrics that drive business success.

### Primary Business Questions:
- How is our revenue trending year-over-year and month-over-month?
- Which product categories are performing best?
- How does performance vary by geographic region?
- What is our customer satisfaction and delivery performance?
- How do our key business metrics compare to previous periods?

### Key Performance Indicators (KPIs):
- **Revenue Growth**: Year-over-year revenue change
- **Average Order Value (AOV)**: Mean order value trends
- **Order Volume**: Number of completed orders
- **Customer Satisfaction**: Average review scores
- **Delivery Performance**: Average delivery time and satisfaction correlation

## 2. Data Loading & Configuration {#data-loading}

Configure analysis parameters and load required data modules.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import warnings

from data_loader import EcommerceDataLoader, get_data_dictionary
from business_metrics import (
    calculate_revenue_metrics, calculate_monthly_growth, calculate_average_order_value,
    calculate_order_volume_metrics, calculate_product_category_performance,
    calculate_geographic_performance, calculate_customer_experience_metrics,
    calculate_order_status_distribution, generate_business_summary
)

warnings.filterwarnings('ignore')

# Set styling for consistent visualizations
plt.style.use('seaborn-v0_8')
color_palette = ['#2E86AB', '#A23B72', '#F18F01', '#C73E1D', '#592E83']
sns.set_palette(color_palette)

# Configuration - Modify these parameters to analyze different time periods
ANALYSIS_CONFIG = {
    'current_year': 2023,
    'previous_year': 2022,
    'analysis_month': None,  # Set to specific month (1-12) or None for full year
    'data_path': 'ecommerce_data'
}

print("Configuration:")
for key, value in ANALYSIS_CONFIG.items():
    print(f"  {key}: {value}")

## 3. Data Dictionary {#data-dictionary}

Understanding our data structure and business terminology.

In [ ]:
data_dict = get_data_dictionary()

for dataset, columns in data_dict.items():
    print(f"\n=== {dataset.upper()} DATASET ===")
    for column, description in columns.items():
        print(f"  {column}: {description}")

## 4. Data Preparation & Quality Assessment {#data-preparation}

Load and prepare data for analysis while assessing data quality.

In [ ]:
# Initialize data loader and load raw data
loader = EcommerceDataLoader(ANALYSIS_CONFIG['data_path'])
raw_data = loader.load_raw_data()

print("\nData Quality Assessment:")
quality_issues = loader.validate_data_quality()
for dataset, issues in quality_issues.items():
    if issues:
        print(f"  {dataset}: {', '.join(issues)}")
    else:
        print(f"  {dataset}: No issues detected")

In [ ]:
# Create comprehensive analysis dataset
analysis_data = loader.create_analysis_dataset(
    year=None,  # Load all years for comparison
    include_geographic=True,
    include_product_info=True,
    include_reviews=True
)

print(f"Analysis dataset shape: {analysis_data.shape}")
print(f"Date range: {analysis_data['order_purchase_timestamp'].min()} to {analysis_data['order_purchase_timestamp'].max()}")
print(f"Years available: {sorted(analysis_data['year'].unique())}")
print(f"Order statuses: {analysis_data['order_status'].value_counts().to_dict()}")

## 5. Revenue Analysis {#revenue-analysis}

Comprehensive analysis of revenue performance including year-over-year growth and monthly trends.

In [ ]:
# Filter data for current analysis period
if ANALYSIS_CONFIG['analysis_month']:
    current_data = loader.filter_by_date_range(
        analysis_data, 
        year=ANALYSIS_CONFIG['current_year'], 
        month=ANALYSIS_CONFIG['analysis_month']
    )
    previous_data = loader.filter_by_date_range(
        analysis_data, 
        year=ANALYSIS_CONFIG['previous_year'], 
        month=ANALYSIS_CONFIG['analysis_month']
    )
    period_label = f"{ANALYSIS_CONFIG['current_year']}-{ANALYSIS_CONFIG['analysis_month']:02d}"
else:
    current_data = loader.filter_by_date_range(analysis_data, year=ANALYSIS_CONFIG['current_year'])
    previous_data = loader.filter_by_date_range(analysis_data, year=ANALYSIS_CONFIG['previous_year'])
    period_label = str(ANALYSIS_CONFIG['current_year'])

# Calculate revenue metrics
revenue_metrics = calculate_revenue_metrics(
    analysis_data, 
    ANALYSIS_CONFIG['current_year'], 
    ANALYSIS_CONFIG['previous_year']
)

print(f"=== REVENUE ANALYSIS FOR {period_label} ===")
print(f"Current Year Revenue: ${revenue_metrics['current_year_revenue']:,.2f}")
print(f"Previous Year Revenue: ${revenue_metrics['previous_year_revenue']:,.2f}")
print(f"Year-over-Year Growth: {revenue_metrics['revenue_growth_percent']:.2f}%")

In [ ]:
# Monthly revenue trend analysis
monthly_growth = calculate_monthly_growth(analysis_data, ANALYSIS_CONFIG['current_year'])

# Create revenue trend visualization
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=[f'Monthly Revenue Trend - {ANALYSIS_CONFIG["current_year"]}', 
                   f'Month-over-Month Growth Rate - {ANALYSIS_CONFIG["current_year"]}'],
    vertical_spacing=0.1
)

# Monthly revenue
monthly_revenue = current_data.groupby('month')['price'].sum().reset_index()
fig.add_trace(
    go.Scatter(
        x=monthly_revenue['month'], 
        y=monthly_revenue['price'],
        mode='lines+markers',
        name='Monthly Revenue',
        line=dict(color='#2E86AB', width=3)
    ),
    row=1, col=1
)

# Monthly growth rate
growth_data = monthly_growth.dropna().reset_index()
fig.add_trace(
    go.Bar(
        x=growth_data['month'], 
        y=growth_data['price'] * 100,
        name='Growth Rate (%)',
        marker_color=['#C73E1D' if x < 0 else '#2E86AB' for x in growth_data['price']]
    ),
    row=2, col=1
)

fig.update_layout(
    height=600,
    title_text=f'Revenue Performance Analysis - {ANALYSIS_CONFIG["current_year"]}',
    title_x=0.5
)

fig.update_xaxes(title_text="Month", row=2, col=1)
fig.update_yaxes(title_text="Revenue ($)", row=1, col=1)
fig.update_yaxes(title_text="Growth Rate (%)", row=2, col=1)

fig.show()

print(f"\nAverage monthly growth rate: {monthly_growth.mean() * 100:.2f}%")
print(f"Best performing month: {monthly_revenue.loc[monthly_revenue['price'].idxmax(), 'month']}")
print(f"Highest monthly revenue: ${monthly_revenue['price'].max():,.2f}")

## 6. Product Performance Analysis {#product-analysis}

Analysis of product category performance and contribution to overall revenue.

In [ ]:
# Calculate product category performance
category_performance = calculate_product_category_performance(
    current_data, 
    raw_data['products']
)

print(f"=== PRODUCT CATEGORY PERFORMANCE - {period_label} ===")
print(category_performance.head(10))

# Calculate category contribution percentages
total_revenue = category_performance['total_revenue'].sum()
category_performance['revenue_share'] = (category_performance['total_revenue'] / total_revenue * 100).round(2)

print(f"\nTop 5 Categories by Revenue Share:")
for idx, row in category_performance.head(5).iterrows():
    print(f"  {idx}: ${row['total_revenue']:,.2f} ({row['revenue_share']:.1f}%)")

In [ ]:
# Create product category visualization
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{"type": "bar"}, {"type": "pie"}]],
    subplot_titles=['Revenue by Product Category', 'Revenue Distribution'],
    horizontal_spacing=0.15
)

# Bar chart for top 10 categories
top_categories = category_performance.head(10)
fig.add_trace(
    go.Bar(
        x=top_categories['total_revenue'],
        y=top_categories.index,
        orientation='h',
        name='Revenue',
        marker_color='#2E86AB'
    ),
    row=1, col=1
)

# Pie chart for top 7 categories + others
top_7 = category_performance.head(7)
others_revenue = category_performance.iloc[7:]['total_revenue'].sum()

pie_labels = list(top_7.index) + ['Others']
pie_values = list(top_7['total_revenue']) + [others_revenue]

fig.add_trace(
    go.Pie(
        labels=pie_labels,
        values=pie_values,
        name="Revenue Share",
        textinfo='label+percent',
        textposition='auto',
        marker_colors=color_palette + ['#95A5A6']
    ),
    row=1, col=2
)

fig.update_layout(
    height=500,
    title_text=f'Product Category Analysis - {period_label}',
    title_x=0.5
)

fig.update_xaxes(title_text="Revenue ($)", row=1, col=1)
fig.update_yaxes(title_text="Product Category", row=1, col=1)

fig.show()

## 7. Geographic Performance Analysis {#geographic-analysis}

Revenue performance analysis by geographic regions (states).

In [ ]:
# Calculate geographic performance
geographic_performance = calculate_geographic_performance(
    current_data,
    raw_data['orders'],
    raw_data['customers']
)

print(f"=== GEOGRAPHIC PERFORMANCE - {period_label} ===")
print("Top 10 States by Revenue:")
print(geographic_performance.head(10))

# Prepare data for choropleth map
geo_data_for_map = geographic_performance.reset_index()
geo_data_for_map.columns = ['customer_state', 'total_revenue', 'total_orders', 'avg_order_value']

In [ ]:
# Create geographic visualizations
fig = make_subplots(
    rows=2, cols=1,
    specs=[[{"type": "geo"}], [{"type": "bar"}]],
    subplot_titles=[f'Revenue by State - {period_label}', 'Top 15 States by Revenue'],
    vertical_spacing=0.15
)

# Choropleth map
fig.add_trace(
    go.Choropleth(
        locations=geo_data_for_map['customer_state'],
        z=geo_data_for_map['total_revenue'],
        locationmode='USA-states',
        colorscale='Reds',
        colorbar_title="Revenue ($)",
        name="Revenue"
    ),
    row=1, col=1
)

# Bar chart for top 15 states
top_15_states = geographic_performance.head(15)
fig.add_trace(
    go.Bar(
        x=top_15_states.index,
        y=top_15_states['total_revenue'],
        name='Revenue',
        marker_color='#A23B72'
    ),
    row=2, col=1
)

fig.update_geos(
    scope='usa',
    showlakes=True,
    lakecolor='rgb(255, 255, 255)'
)

fig.update_layout(
    height=800,
    title_text=f'Geographic Revenue Analysis - {period_label}',
    title_x=0.5
)

fig.update_xaxes(title_text="State", row=2, col=1)
fig.update_yaxes(title_text="Revenue ($)", row=2, col=1)

fig.show()

print(f"\nGeographic Insights:")
print(f"  Highest revenue state: {geographic_performance.index[0]} (${geographic_performance.iloc[0]['total_revenue']:,.2f})")
print(f"  States with revenue data: {len(geographic_performance)}")
print(f"  Average revenue per state: ${geographic_performance['total_revenue'].mean():,.2f}")

## 8. Customer Experience Analysis {#customer-experience}

Analysis of customer satisfaction, delivery performance, and their correlation.

In [ ]:
# Calculate customer experience metrics
cx_metrics = calculate_customer_experience_metrics(
    current_data,
    raw_data['reviews']
)

print(f"=== CUSTOMER EXPERIENCE METRICS - {period_label} ===")
print(f"Average Delivery Time: {cx_metrics['avg_delivery_days']:.1f} days")
print(f"Average Review Score: {cx_metrics['avg_review_score']:.2f}/5.0")
print(f"\nSatisfaction by Delivery Speed:")
for category, score in cx_metrics['delivery_satisfaction_by_speed'].items():
    print(f"  {category}: {score:.3f}/5.0")

# Calculate order status distribution
order_status_dist = calculate_order_status_distribution(
    raw_data['orders'], 
    ANALYSIS_CONFIG['current_year']
)

print(f"\nOrder Status Distribution - {ANALYSIS_CONFIG['current_year']}:")
for status, percentage in order_status_dist.items():
    print(f"  {status}: {percentage:.1%}")

In [ ]:
# Create customer experience visualizations
fig = make_subplots(
    rows=2, cols=2,
    specs=[[{"type": "bar"}, {"type": "pie"}],
           [{"type": "histogram"}, {"type": "bar"}]],
    subplot_titles=['Satisfaction by Delivery Speed', 'Order Status Distribution',
                   'Review Score Distribution', 'Average Order Value Metrics'],
    horizontal_spacing=0.15,
    vertical_spacing=0.15
)

# Satisfaction by delivery speed
delivery_cats = list(cx_metrics['delivery_satisfaction_by_speed'].keys())
delivery_scores = list(cx_metrics['delivery_satisfaction_by_speed'].values())

fig.add_trace(
    go.Bar(
        x=delivery_cats,
        y=delivery_scores,
        name='Review Score',
        marker_color='#F18F01'
    ),
    row=1, col=1
)

# Order status pie chart
fig.add_trace(
    go.Pie(
        labels=order_status_dist.index,
        values=order_status_dist.values,
        name="Order Status",
        textinfo='label+percent'
    ),
    row=1, col=2
)

# Review score distribution
review_dist = current_data['review_score'].value_counts().sort_index()
fig.add_trace(
    go.Bar(
        x=review_dist.index,
        y=review_dist.values,
        name='Review Count',
        marker_color='#592E83'
    ),
    row=2, col=1
)

# AOV comparison
aov_metrics = calculate_average_order_value(
    analysis_data, 
    ANALYSIS_CONFIG['current_year'], 
    ANALYSIS_CONFIG['previous_year']
)

fig.add_trace(
    go.Bar(
        x=[str(ANALYSIS_CONFIG['previous_year']), str(ANALYSIS_CONFIG['current_year'])],
        y=[aov_metrics['previous_year_aov'], aov_metrics['current_year_aov']],
        name='Average Order Value',
        marker_color=['#A23B72', '#2E86AB']
    ),
    row=2, col=2
)

fig.update_layout(
    height=700,
    title_text=f'Customer Experience Dashboard - {period_label}',
    title_x=0.5,
    showlegend=False
)

fig.update_yaxes(title_text="Review Score", row=1, col=1)
fig.update_yaxes(title_text="Order Count", row=2, col=1)
fig.update_yaxes(title_text="AOV ($)", row=2, col=2)
fig.update_xaxes(title_text="Delivery Speed", row=1, col=1)
fig.update_xaxes(title_text="Review Score", row=2, col=1)
fig.update_xaxes(title_text="Year", row=2, col=2)

fig.show()

print(f"\nAOV Change: {aov_metrics['aov_growth_percent']:.2f}%")

## 9. Summary of Key Insights {#summary}

Comprehensive business performance summary and actionable insights.

In [ ]:
# Calculate all key metrics for summary
order_metrics = calculate_order_volume_metrics(
    analysis_data, 
    ANALYSIS_CONFIG['current_year'], 
    ANALYSIS_CONFIG['previous_year']
)

# Generate comprehensive business summary
business_summary = generate_business_summary(
    revenue_metrics, aov_metrics, order_metrics, cx_metrics
)

print(f"=== BUSINESS PERFORMANCE SUMMARY - {period_label} ===")
print("\n📊 REVENUE PERFORMANCE")
print(f"  Current Period Revenue: {business_summary['revenue_summary']['current_revenue']:,.2f}")
print(f"  Revenue Growth: {business_summary['revenue_summary']['revenue_growth']}")

print("\n🛒 ORDER METRICS")
print(f"  Total Orders: {business_summary['order_summary']['current_orders']:,}")
print(f"  Average Order Value: {business_summary['order_summary']['current_aov']}")
print(f"  Order Growth: {business_summary['order_summary']['order_growth']}")

print("\n🎯 CUSTOMER EXPERIENCE")
print(f"  Average Delivery Time: {business_summary['customer_experience']['avg_delivery_days']}")
print(f"  Customer Satisfaction: {business_summary['customer_experience']['avg_satisfaction']}")

print("\n🏆 TOP PERFORMERS")
print(f"  Best Product Category: {category_performance.index[0]} (${category_performance.iloc[0]['total_revenue']:,.2f})")
print(f"  Highest Revenue State: {geographic_performance.index[0]} (${geographic_performance.iloc[0]['total_revenue']:,.2f})")
print(f"  Best Monthly Performance: Month {monthly_revenue.loc[monthly_revenue['price'].idxmax(), 'month']} (${monthly_revenue['price'].max():,.2f})")

In [ ]:
# Create executive summary table
summary_data = {
    'Metric': [
        'Total Revenue', 'Revenue Growth (YoY)', 'Total Orders', 'Order Growth (YoY)',
        'Average Order Value', 'AOV Growth (YoY)', 'Avg Delivery Time', 'Customer Satisfaction',
        'Order Completion Rate', 'Top Product Category', 'Top Revenue State'
    ],
    'Value': [
        f"${revenue_metrics['current_year_revenue']:,.0f}",
        f"{revenue_metrics['revenue_growth_percent']:.1f}%",
        f"{order_metrics['current_year_orders']:,}",
        f"{order_metrics['order_growth_percent']:.1f}%",
        f"${aov_metrics['current_year_aov']:.0f}",
        f"{aov_metrics['aov_growth_percent']:.1f}%",
        f"{cx_metrics['avg_delivery_days']:.1f} days",
        f"{cx_metrics['avg_review_score']:.2f}/5.0",
        f"{order_status_dist['delivered']:.1%}",
        category_performance.index[0],
        geographic_performance.index[0]
    ]
}

summary_df = pd.DataFrame(summary_data)
print(f"\n=== EXECUTIVE SUMMARY TABLE - {period_label} ===")
print(summary_df.to_string(index=False))

### Key Recommendations

Based on the analysis results, here are the recommended actions:

**Revenue Growth:**
- Focus on months showing declining growth to understand seasonal patterns
- Investigate factors contributing to revenue changes year-over-year

**Product Strategy:**
- Invest more in top-performing product categories
- Analyze underperforming categories for improvement opportunities
- Consider expanding successful product lines

**Geographic Expansion:**
- Focus marketing efforts on high-performing states
- Investigate expansion opportunities in underrepresented regions
- Optimize logistics for top revenue-generating areas

**Customer Experience:**
- Work on reducing delivery times, especially for categories showing lower satisfaction
- Implement targeted improvements for delivery speed categories with lower review scores
- Monitor and improve order fulfillment processes to maintain high completion rates

---

**Note:** This analysis is configurable for different time periods. Modify the `ANALYSIS_CONFIG` parameters at the top of the notebook to analyze different months, years, or date ranges.